In [38]:
import io
import sys

from google.cloud import storage

sys.path.append("..")
from function.retriever import initialize_retriever

retriever = initialize_retriever()

metadatas = [entry.metadata for entry in retriever.vectorstore.docstore._dict.values()]
all_stored_venues = set(_["company"] for _ in metadatas)
all_stored_venues

Loaded existing FAISS index from faiss_db


{'dummy'}

In [2]:
from pathlib import Path

from function.cloud import list_files

all_venues = set([Path(path).stem for path in list_files(filter=r"venues/")])

unprocessed_venues = all_venues - all_stored_venues
venue = list(unprocessed_venues)[1:10]
venue


['The Thursday Club',
 'Santa Maria Country Club',
 'The Peninsula Beverly Hills',
 'Horton Grand Hotel',
 'Marina Del Rey Hotel',
 'The Crossings at Carlsbad',
 'The Gardens at Peacock Farms',
 'L_Auberge del mar',
 'Los Angeles Athletic Club']

In [39]:
import os

from dotenv import load_dotenv

from function.retriever import (
    add_documents_to_retriever,
    load_venue_metadata,
    preprocess_document,
    preprocess_documents,
)

In [9]:
venue_metadata = load_venue_metadata()
preprocess_document(venue[0], venue_metadata)

searching for The Thursday Club.pdf on google cloud...
downloading The Thursday Club.pdf from google cloud...
sending The Thursday Club.pdf to Adobe...
extracting text from pdf...
2024-12-22 11:45:24 unzip file
2024-12-22 11:45:24 open json file
2024-12-22 11:45:24 extract text
generating image descriptions for The Thursday Club.pdf
Generating image descriptions...
   (1/37) /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart8.png
skipping /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart8.png
   (2/37) /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart9.png
skipping /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart9.png
   (3/37) /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart22.png
   (4/37) /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/tmp3fkue2a2/figures/fileoutpart36.png
skipping /var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/

{'doc_id': 'c1eb74e6-9724-4e5e-a786-4f87f7df2d5f',
 'text_content': 'T/r e \nTHURSDAY CLUB Wedding Guide \n1224 Santa Barbara Street [ San Diego, CA 92107 6 t 9 , 2 2 4 , 5 2 6 4 I m a n a g e r @ t h e t h u r s d a y c 1 u b , o r g_ www.TheThursdayClub.org \nr 8 Hours Facility Rental \n** 2 Hour set up • 5 Hour Event • 1 Hour clean up Includes our inventory of tables and chairs \nSaturday ] $6800 \nFriday & Sunday I $5800 Monday -Thursday I $4800 \nAdditional hours can be added to the beginning \nWhite string market lights I $300 \nCustom color up-lights (16 up-lights total) I $300 Full lighting package includes string lights & up-lights I $500 \nr1 50% of the rental fee & a $1000 security deposit are collected when the reservation is made. The balance of the rental fee is collected 90 days before the event. The security deposit is a refundable ■ deposit to cover damagesital honor l«tel@l timd \nupdated 10-17-24 \nHours of Use \nThe Thursday Club is located in the beautiful Sunset C

In [28]:
info = preprocess_documents(venue[1:3], venue_metadata)
# PERSIST_DIRECTORY: str = os.getenv("DATABASE_DIR")
PERSIST_DIRECTORY = "notebooks/faiss_db"

add_documents_to_retriever(info, retriever, venue_metadata)
retriever.vectorstore.save_local(PERSIST_DIRECTORY)

Processed document: Santa Maria Country Club
Processed document: The Peninsula Beverly Hills


In [46]:
from function.secrets import secrets

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=secrets.OPENAI_API_KEY.get_secret_value(),
)
# if not os.path.exists(PERSIST_DIRECTORY):
# print("Fetching database from the cloud...")
# download_faiss_from_cloud(bucket.blob("faiss_db"), PERSIST_DIRECTORY)
# Try to load

vectorstore = FAISS.load_local(
    "faiss_db", embedding_model, allow_dangerous_deserialization=True
)

In [50]:
from dotenv import find_dotenv

path = find_dotenv()
PROJECT_ROOT = os.path.dirname(path)

'/Users/lanchu/Documents/projects/001-wedding-venue'

In [48]:
os.path.join(os.getcwd(), os.getenv("DATABASE_DIR"))

'/Users/lanchu/Documents/projects/001-wedding-venue/notebooks/faiss_db'

In [47]:
for item in vectorstore.docstore._dict.values():
    print(item.metadata["company"], item.metadata["type"])

dummy text
The Thursday Club text
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
The Thursday Club image
Santa Maria Country Club text
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Country Club image
Santa Maria Cou

In [37]:
retriever.vectorstore.docstore._dict

{'58eeee0b-08f4-4026-b2a5-03f4abb80be0': Document(metadata={'company': 'dummy', 'type': 'text'}, page_content='dummy document')}

In [40]:
PERSIST_DIRECTORY = "notebooks/faiss_db"
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
)
vectorstore = FAISS.load_local(
    PERSIST_DIRECTORY, embedding_model, allow_dangerous_deserialization=True
)

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/io.cpp:68: Error: 'f' failed: could not open notebooks/faiss_db/index.faiss for reading: No such file or directory